# 1. Convert Polygon Annotations to Segmentation Masks

### 1.1. Parsing the Annotations
We'll start by extracting the polygon coordinates for the OD and ID from the annotations.

In [1]:
import os
import numpy as np

def parse_annotations(annotation_path, image_shape):
    """
    Parse the annotations and return the polygons for OD and ID.
    :param annotation_path: Path to the annotation file.
    :param image_shape: Shape of the image.
    :return: Polygons for OD and ID.
    """
    height, width, _ = image_shape

    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    od_polygon = []
    id_polygon = []

    for line in lines:
        line = line.strip()
        elements = line.split()

        # Extract class label
        label = int(elements[0])
        # Extract x,y coordinates, scale by image width and height
        coordinates = [(int(float(elements[i]) * width), int(float(elements[i+1]) * height)) for i in range(1, len(elements), 2)]
        
        # Check if the coordinates are being parsed correctly
        print("Coordinates for label", label, ":", coordinates)

        if label == 4:  # OD
            od_polygon.extend(coordinates)
        elif label == 1:  # ID
            id_polygon.extend(coordinates)

    return np.array(od_polygon), np.array(id_polygon)




### 1.2. Rasterizing Polygons
Given the extracted polygons, we'll convert them into binary masks using OpenCV.

In [2]:
import cv2

def create_mask_from_polygon(img_shape, polygon):
    """
    Rasterizes the polygon into a binary mask.
    :param img_shape: Shape of the associated image.
    :param polygon: Polygon coordinates.
    :return: Binary mask.
    """
    mask = np.zeros(img_shape, np.uint8)
    cv2.fillPoly(mask, [polygon], 255)
    return mask


### 1.3. Creating Coating Mask
Now, we'll generate the mask representing the coating region by subtracting the ID mask from the OD mask.

In [10]:
def create_coating_mask(od_mask, id_mask):
    """
    Generates the mask for the coating region.
    :param od_mask: Mask of the Outer Diameter.
    :param id_mask: Mask of the Inner Diameter.
    :return: Binary mask representing the coating.
    """
    return cv2.subtract(id_mask, od_mask)


### 1.4. Save the Resulting Masks
Finally, we'll save the generated masks to the desired location.

In [4]:
def save_mask(mask, filename, output_path):
    """
    Saves the mask to the specified location.
    :param mask: Binary mask to save.
    :param filename: Original filename to use for naming the mask.
    :param output_path: Directory where to save the mask.
    """
    mask_filename = os.path.join(output_path, filename.replace('.png', '_mask.png'))
    cv2.imwrite(mask_filename, mask)


Visualization for debugging

In [5]:
def visualize_polygon(image, od_polygon, id_polygon):
    """
    Display an image with the OD and ID polygons overlayed.
    :param image: Source image.
    :param od_polygon: Polygon for OD.
    :param id_polygon: Polygon for ID.
    :return: Image with overlayed polygons.
    """
    overlayed_image = image.copy()
    cv2.polylines(overlayed_image, [od_polygon], isClosed=True, color=(255, 0, 0), thickness=2)
    cv2.polylines(overlayed_image, [id_polygon], isClosed=True, color=(0, 255, 0), thickness=2)
    return overlayed_image


### 1.5. Put it all together and save the masks to a new directory within this project
We will take the images/annotations from the YOLO project and save the masks to this project directory

In [11]:
images_path = '/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/yolo/capture_data/2023-08-22/images'
annotations_path = '/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/yolo/capture_data/2023-08-22/txt'
output_path = '/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/segmentation_masks'

if not os.path.exists(output_path):
    os.makedirs(output_path)

for filename in os.listdir(annotations_path):
    if filename.endswith(".txt"):
        # Load associated image to get its shape
        img_name = filename.replace('.txt', '.png')
        img = cv2.imread(os.path.join(images_path, img_name))
        
        # Extract and scale OD and ID polygons
        od_polygon, id_polygon = parse_annotations(os.path.join(annotations_path, filename), img.shape)
        
        print("OD Polygon:", od_polygon)
        print("ID Polygon:", id_polygon)


        
        # Visualize polygons
        # overlayed_img = visualize_polygon(img, od_polygon, id_polygon)
        # cv2.imshow('Overlayed Image', overlayed_img)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        
        # Convert polygons to binary masks
        od_mask = create_mask_from_polygon(img.shape[:2], od_polygon)
        id_mask = create_mask_from_polygon(img.shape[:2], id_polygon)

        print(f'Number of non-zero pixels in OD mask: {np.count_nonzero(od_mask)}')
        print(f'Number of non-zero pixels in ID mask: {np.count_nonzero(id_mask)}')

        
        # Generate the coating mask
        coating_mask = create_coating_mask(od_mask, id_mask)

        print(f'Number of non-zero pixels in coating mask: {np.count_nonzero(coating_mask)}')

        
        # Save the mask
        save_mask(coating_mask, img_name, output_path)

        print(f'Processed {img_name}')



Coordinates for label 4 : [(1182, 903), (1140, 936), (1121, 985), (1125, 1049), (1147, 1090), (1179, 1124), (1218, 1150), (1277, 1159), (1314, 1159), (1393, 1099), (1415, 1071), (1392, 1067), (1414, 1033), (1418, 991), (1397, 949), (1376, 921), (1329, 881), (1303, 879), (1295, 879), (1285, 870), (1238, 868), (1230, 876), (1202, 895), (1180, 905)]
Coordinates for label 1 : [(1096, 862), (1082, 874), (1076, 875), (1076, 883), (1065, 899), (1053, 913), (1051, 927), (1046, 937), (1046, 972), (1035, 984), (1034, 1018), (1041, 1033), (1037, 1054), (1055, 1101), (1073, 1139), (1096, 1162), (1128, 1197), (1157, 1216), (1201, 1222), (1234, 1234), (1257, 1235), (1273, 1240), (1289, 1234), (1321, 1236), (1332, 1233), (1329, 1228), (1343, 1228), (1362, 1227), (1369, 1214), (1387, 1216), (1422, 1193), (1467, 1148), (1492, 1105), (1499, 1069), (1509, 1015), (1503, 959), (1493, 932), (1467, 878), (1450, 866), (1425, 837), (1397, 827), (1379, 811), (1355, 798), (1328, 801), (1306, 794), (1234, 792), (

# 2. Augment images to increase dataset size
Our dataset contains ~250 images. We will perform 90°, 180°, 270° Rotations, vertical flip and horizontal flip. This will increase our dataset to ~1500 images

In [16]:
def rotate_image(image, angle):
    """
    Rotates an image by the given angle.

    Parameters:
    - image: numpy array, the image to be rotated
    - angle: int, rotation angle in degrees

    Returns:
    - Rotated image as a numpy array
    """
    
    # Get the dimensions of the image
    height, width = image.shape[:2]
    
    # Define the center of the image
    center = (width/2, height/2)
    
    # Compute the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
    
    # Perform the rotation on the image using the rotation matrix
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    
    return rotated_image


In [18]:
import cv2
import numpy as np
import os

def augment_image_and_mask(image, mask):
    # List to store augmented images and masks
    augmented_images = [image]
    augmented_masks = [mask]
    
    # 90°, 180°, 270° Rotations
    # rotated_images = [image]
    # rotated_masks = [mask]

    for angle in [90, 180, 270]:
        augmented_images.append(rotate_image(image, angle))
        augmented_masks.append(rotate_image(mask, angle))


    # Horizontal Flip
    augmented_images.append(cv2.flip(image, 1))
    augmented_masks.append(cv2.flip(mask, 1))

    # Vertical Flip
    augmented_images.append(cv2.flip(image, 0))
    augmented_masks.append(cv2.flip(mask, 0))

    return augmented_images, augmented_masks

images_path = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/yolo/capture_data/2023-08-22/images"
masks_path = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/segmentation_masks"
augmented_images_path = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/augmented/images"
augmented_masks_path = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/augmented/masks"    

for filename in os.listdir(images_path):
    if filename.endswith(".png"):
        image = cv2.imread(os.path.join(images_path, filename))
        mask_name = filename.replace('.png', '_mask.png')
        mask = cv2.imread(os.path.join(masks_path, mask_name), cv2.IMREAD_GRAYSCALE)

        if image is None:
            print(f"Failed to load image: {filename}")
            continue
        if mask is None:
            print(f"Failed to load mask: {mask_name}")
            continue
        
        augmented_images, augmented_masks = augment_image_and_mask(image, mask)
        
        # Save the augmented images and masks
        for idx, (aug_img, aug_mask) in enumerate(zip(augmented_images, augmented_masks)):
            if aug_img is None or aug_img.size == 0:
                print(f"Augmented image is empty for: {filename}, augmentation index: {idx}")
                continue
            if aug_mask is None or aug_mask.size == 0:
                print(f"Augmented mask is empty for: {mask_name}, augmentation index: {idx}")
                continue
            aug_img_name = filename.replace('.png', f'_aug{idx}.png')
            aug_mask_name = mask_name.replace('_mask.png', f'_aug{idx}_mask.png')
            
            cv2.imwrite(os.path.join(augmented_images_path, aug_img_name), aug_img)
            cv2.imwrite(os.path.join(augmented_masks_path, aug_mask_name), aug_mask)


# 3. Split into train/test/val
80/10/10

In [20]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths to your augmented images and masks
image_directory = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/augmented/images"
mask_directory = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/augmented/masks"

# Create main directory
main_directory = '/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset'
os.makedirs(main_directory, exist_ok=True)

# Create subdirectories for training, validation, and testing
training_dir = os.path.join(main_directory, 'training')
validation_dir = os.path.join(main_directory, 'validation')
testing_dir = os.path.join(main_directory, 'testing')

for dir in [training_dir, validation_dir, testing_dir]:
    os.makedirs(os.path.join(dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dir, 'masks'), exist_ok=True)

# Get list of image and mask files
image_files = [f for f in os.listdir(image_directory) if f.endswith('.png')]
mask_files = [f for f in os.listdir(mask_directory) if f.endswith('_mask.png')]

# Split into training, validation, and testing
train_files, test_files = train_test_split(image_files, test_size=0.20, random_state=42)
val_files, test_files = train_test_split(test_files, test_size=0.50, random_state=42)

# Function to copy files
def copy_files(files, src_image_dir, src_mask_dir, dest_image_dir, dest_mask_dir):
    for filename in files:
        shutil.copy(os.path.join(src_image_dir, filename), os.path.join(dest_image_dir, filename))
        mask_file = filename.replace('.png', '_mask.png')
        shutil.copy(os.path.join(src_mask_dir, mask_file), os.path.join(dest_mask_dir, mask_file))

# Copy files to respective directories
copy_files(train_files, image_directory, mask_directory, os.path.join(training_dir, 'images'), os.path.join(training_dir, 'masks'))
copy_files(val_files, image_directory, mask_directory, os.path.join(validation_dir, 'images'), os.path.join(validation_dir, 'masks'))
copy_files(test_files, image_directory, mask_directory, os.path.join(testing_dir, 'images'), os.path.join(testing_dir, 'masks'))

print("Dataset organized successfully!")


Dataset organized successfully!


# 4. Train U-Net Model

### 4.1. Downsize images
512 x 512

In [21]:
import os
import cv2

def resize_images_in_directory(directory, size=(512, 512)):
    """
    Resize images in a given directory to the specified size.
    
    :param directory: Path to the directory containing images/masks.
    :param size: Tuple (width, height) defining the new size. Default is (512, 512).
    """
    for subdir, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                file_path = os.path.join(subdir, file)
                img = cv2.imread(file_path)
                if img is None:
                    print(f"Failed to read {file_path}. Skipping...")
                    continue
                resized_img = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
                cv2.imwrite(file_path, resized_img)

# Resize images and masks in the training/testing/validation directories
dirs = [
    "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset/training/images",
    "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset/training/masks",
    "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset/testing/images",
    "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset/testing/masks",
    "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset/validation/images",
    "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset/validation/masks"
]

for directory in dirs:
    resize_images_in_directory(directory)


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow_addons.metrics import MeanIoU

# Dataset paths
TRAIN_IMG_DIR = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset/training/images"
TRAIN_MASK_DIR = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset/training/masks"
TEST_IMG_DIR = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset/testing/images"
TEST_MASK_DIR = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset/testing/masks"
VAL_IMG_DIR = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset/validation/images"
VAL_MASK_DIR = "/Users/tylerhouchin/Desktop/Mediprint/lens_inspection/cnn_segmentation/dataset/validation/masks"

# Hyperparameters
IMG_SIZE = (512, 512)
BATCH_SIZE = 8
EPOCHS = 50
LEARNING_RATE = 1e-4

# U-Net architecture
def build_unet(input_shape):
    inputs = Input(input_shape)
    
    # Contracting path
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    
    # Expanding path
    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    # Final output
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    return Model(inputs=inputs, outputs=outputs)

# Load dataset
def load_data(img_dir, mask_dir):
    img_files = sorted([os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith(".png")])
    mask_files = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith(".png")])

    images = [img_to_array(load_img(img_path, target_size=IMG_SIZE)) for img_path in img_files]
    masks = [img_to_array(load_img(mask_path, target_size=IMG_SIZE, color_mode="grayscale")) for mask_path in mask_files]

    return np.array(images) / 255.0, np.array(masks) / 255.0

# Train the model
X_train, y_train = load_data(TRAIN_IMG_DIR, TRAIN_MASK_DIR)
X_val, y_val = load_data(VAL_IMG_DIR, VAL_MASK_DIR)

model = build_unet((*IMG_SIZE, 3))
model.compile(optimizer=Adam(LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy', MeanIoU(num_classes=2)])

callbacks = [
    ModelCheckpoint('best_weights.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min', verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
]

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks)

print("Model trained successfully!")
